In [3]:
import spacy
nlp = spacy.load('pl_spacy_model_morfeusz')

In [4]:
# Print the names of the pipeline components
print(nlp.pipe_names)

# Print the full pipeline of (name, component) tuples
print(nlp.pipeline)
# The Preprocessor is not listed as it is a substitute for the tokenizer
# The pipeline does not include the tagger, as it is contained within the Preprocessor

['parser', 'ner']
[('parser', <spacy.pipeline.pipes.DependencyParser object at 0x00000128956EDC48>), ('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x00000128956EDCA8>)]


In [5]:
# List the tokens and UPOS tags
doc = nlp("Granice mojego języka oznaczają granice mojego świata") # ~Wittgenstein
for token in doc:
    print(token.text, token.lemma_, token.pos_)

Granice Granice NOUN
mojego mój ADJ
języka język NOUN
oznaczają oznaczać VERB
granice granica NOUN
mojego mój ADJ
świata świat NOUN


In [12]:
# In pl_spacy_model_morfeusz token.tag_ stores an 'extended tag'

# The extended tag stores a POS tag, and all the morphosyntactic features (e.g. case or grammatical gender)
# for which Morfeusz was unambiguous.
# If you are only interested in the NKJP POS tag, you can just split after the first colon
for token in doc:
    print('\t'.join([token.orth_, token.tag_.split(":")[0], token.tag_]))

Granice	SUBST	SUBST:pl
mojego	ADJ	ADJ:sg:pos
języka	SUBST	SUBST:sg:gen:m3
oznaczają	FIN	FIN:pl:ter:imperf
granice	SUBST	SUBST:pl:f
mojego	ADJ	ADJ:sg:pos
świata	SUBST	SUBST:sg:gen:m3


In [13]:
# Tokenization, Tagging, Lemmatization and Dependency Parsing in a pretty table
import pandas as pd

attribs = ['orth_', 'lemma_', 'tag_', 'pos_', 'dep_', 'head']
table = [{att:tok.__getattribute__(att) for att in attribs} for tok in doc]
df = pd.DataFrame(table)
print(df[attribs])

       orth_    lemma_               tag_  pos_      dep_       head
0    Granice   Granice           SUBST:pl  NOUN     nsubj  oznaczają
1     mojego       mój         ADJ:sg:pos   ADJ  det:poss     języka
2     języka     język    SUBST:sg:gen:m3  NOUN  nmod:arg    Granice
3  oznaczają  oznaczać  FIN:pl:ter:imperf  VERB      ROOT  oznaczają
4    granice   granica         SUBST:pl:f  NOUN       obj  oznaczają
5     mojego       mój         ADJ:sg:pos   ADJ  det:poss     świata
6     świata     świat    SUBST:sg:gen:m3  NOUN  nmod:arg    granice


In [14]:
# Play with the pos values
for token in doc:
    # Check if the current token is a proper noun
    if token.pos_ == "ADJ":
        # Check if the next token is a verb
        if doc[token.i + 1].pos_ == "NOUN":
            print("Found adjective before a noun:", token.text)

Found adjective before a noun: mojego
Found adjective before a noun: mojego


In [15]:
# DEP visualization
from spacy import displacy
displacy.render(doc, style="dep")

In [16]:
# Parsing multiple sentences
doc = nlp("Prof. Nowak wyszedł z domu spóźniony. Potknął się o próg i upadł. \"Nic się nie stało?\" - pytała sąsiadka.")
for sent in doc.sents:
    print(sent)

Prof. Nowak wyszedł z domu spóźniony.
Potknął się o próg i upadł.
"Nic się nie stało?" - pytała sąsiadka.


In [17]:
displacy.render(doc, style = 'dep', options = {"compact": True})

In [18]:
# Named entities
doc = nlp("Selekcjoner reprezentacji Polski Jerzy Brzęczek \
        powołał piłkarzy na mecze eliminacji mistrzostw Europy 2020,\
        6 września Polacy zagrają w Lublanie ze Słowenią.") # ~Rzeczpospolita 

for entity in doc.ents:
    print(entity.text, entity.label_)

Polski placeName
Jerzy Brzęczek persName
Europy geogName
2020 date
6 września date
Polacy placeName
Lublanie placeName
Słowenią placeName


In [19]:
# NER visualization
displacy.render(doc, style="ent")

In [20]:
# Simple token attributes
for token in doc:
    # Check if the token resembles a number
    if token.like_num:
        print(token)

2020
6
